# W261 Final Project ETL for Development Sample

### Notebook Set-Up

In [1]:
# imports
import time
import numpy as np
import pandas as pd
from pyspark.sql import Row
from pyspark.ml.feature import CountVectorizer
from pyspark.sql import DataFrame

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]
PWD

'/media/notebooks'

In [4]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "w261FinalProject"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

__`REMINDER:`__ If you are running this notebook on the course docker container, you can monitor the progress of your jobs using the Spark UI at: http://localhost:4040/jobs/

## Load Data

In [5]:
original_trainRDD = sc.textFile('data/train.txt')
original_testRDD = sc.textFile('data/test.txt')

## Transformation

### Sample

In [6]:
#change the seed for a different sample
sampleRDD1, sampleRDD2 = original_trainRDD.randomSplit([0.9999,0.0001], seed = 1)
sampleRDD2.cache()

PythonRDD[4] at RDD at PythonRDD.scala:49

In [7]:
ncol = len(sampleRDD2.take(1)[0].split('\t'))
nrow = sampleRDD2.count()
print("This sample contains", str(nrow), "rows.")

This sample contains 4478 rows.


In [8]:
print("This sample contains", str(ncol), "columns.")

This sample contains 40 columns.


In [9]:
def avgFeatures(line):
    
    count = 0
    feats = line.split('\t')[1:]
    
    for feat in feats:
        if feat != '':
            count += 1

    return count

print("There is an average of", str(round(sampleRDD2.map(avgFeatures).mean(),2)), "populated features per observation.")
sampleRDD2.map(avgFeatures).mean()

There is an average of 33.53 populated features per observation.


33.5288075033497

# Put in wide, sparse feature format

In [10]:
def parseCV(line):
    """
    Map record_csv_string --> (features, label)
    """

    # start of categorical features
    col_start = 14
    
    raw_values = line.split('\t')
    label = int(raw_values[0])
    
    # ignore numerics to start
    #numerical_values = list(pd.Series(raw_values[1:14]).apply(pd.to_numeric))
    numericals = []
    for idx, value in enumerate(raw_values[1:col_start]):
        if value != '':
            numericals.append('n' + str(idx) + '_' + str(value))
            
    
    categories = []
    for idx, value in enumerate(raw_values[col_start:]):
        if value != '':
            categories.append('c'+ str(idx) + '_' + str(value))

    return Row(label=label, raw=numericals + categories)


def vectorizeCV(DF):
    
    vectorizer = CountVectorizer()
    cv = CountVectorizer(inputCol="raw", outputCol="features", binary=True)
    
    model = cv.fit(DF)
    result = model.transform(DF)
    num_feats = cv.getVocabSize()
    
    return result


parsedDF = sampleRDD2.map(parseCV).toDF().cache()
vectorizedDF = vectorizeCV(parsedDF)

In [11]:
vectorizedDF.show(truncate=True)

+-----+--------------------+--------------------+
|label|                 raw|            features|
+-----+--------------------+--------------------+
|    0|[n1_4, n2_50, n3_...|(30946,[0,1,2,4,5...|
|    0|[n1_12, n2_20, n3...|(30946,[0,1,2,5,1...|
|    1|[n1_1, n2_1, n4_9...|(30946,[0,1,6,7,1...|
|    0|[n0_8, n1_17, n3_...|(30946,[0,1,4,12,...|
|    1|[n0_6, n1_1, n2_7...|(30946,[0,1,2,4,1...|
|    1|[n1_99, n2_1, n3_...|(30946,[1,2,4,10,...|
|    0|[n0_3, n1_21, n2_...|(30946,[0,1,4,8,1...|
|    0|[n1_2, n2_20, n3_...|(30946,[0,1,3,5,8...|
|    0|[n0_0, n1_144, n4...|(30946,[0,2,3,4,5...|
|    0|[n1_0, n2_5, n4_3...|(30946,[0,2,3,6,1...|
|    0|[n0_0, n1_1, n2_4...|(30946,[0,1,2,3,5...|
|    0|[n0_9, n1_5, n2_1...|(30946,[0,2,3,6,9...|
|    0|[n1_323, n2_2, n3...|(30946,[1,2,14,16...|
|    0|[n0_0, n1_424, n3...|(30946,[0,1,2,4,6...|
|    0|[n0_0, n1_13, n2_...|(30946,[0,1,2,5,6...|
|    0|[n1_180, n2_6, n3...|(30946,[1,2,8,14,...|
|    0|[n1_126, n2_2, n3...|(30946,[0,2,4,6,8...|


In [12]:
vectorizedRDD = vectorizedDF.select(['label', 'features']).rdd.cache()
vectorizedRDD.take(1)

[Row(label=0, features=SparseVector(30946, {0: 1.0, 1: 1.0, 2: 1.0, 4: 1.0, 5: 1.0, 7: 1.0, 10: 1.0, 20: 1.0, 32: 1.0, 122: 1.0, 154: 1.0, 175: 1.0, 214: 1.0, 364: 1.0, 369: 1.0, 495: 1.0, 504: 1.0, 626: 1.0, 634: 1.0, 855: 1.0, 1882: 1.0, 2080: 1.0, 2237: 1.0, 2281: 1.0, 2733: 1.0, 6170: 1.0, 10077: 1.0, 11284: 1.0, 14033: 1.0, 17114: 1.0, 19322: 1.0, 20016: 1.0, 25376: 1.0, 25620: 1.0}))]

In [13]:
# feature data struct tester
num_feats = vectorizedRDD.take(1)[0][1].size
num_feats

30946

In [14]:
vectorizedRDD.map(lambda x: x[0]).mean()

0.25949084412684253

## test prediction making

In [15]:
#sc.addPyFile("./fm_function_tester.py")
#import fm_function_tester as fm
#from fm_function_tester import *

In [15]:
# initialize weights
np.random.seed(24)

b_br = sc.broadcast(0.0)
w_br = sc.broadcast(np.zeros(num_feats, dtype=np.float16))
k_br = sc.broadcast(2)
V_br = sc.broadcast(np.random.normal(0.0, 0.5, (num_feats, k_br.value)))

In [117]:
indices = [1,3,5, 20]
np.dot(V_br.value[indices][:,0]**2, np.array(indices)**2)

163.41736428150125

In [70]:
x = np.array([2,3])
y = np.array([5,6])
np.multiply(x**2,y)

array([20, 54])

In [16]:
import numpy as np
from pyspark.ml.feature import CountVectorizer
from scipy.sparse import csr_matrix

def predict_grad(pair):
    """
        Compute the predicted probability AND return the gradient (?)
        Args:
            pair - records are in (label, sparse feature set) format
        Broadcast:
            b - bias term (scalar)
            w - linear weight vector (array)
            k - number of factors (def=2)
            V - factor matrix of size (d dimensions, k=2 factors)
        Returns:
            predRDD - pair of ([label, predicted probability], feature set)
    """
    
    label = pair[0]
    feats = pair[1]
    
    # start with linear weight dot product
    linear_sum = 0.0
    for i in feats.indices:
        i = int(i)
        linear_sum += w_br.value[i]*feats[i]

    # factor matrix interaction sum
    factor_sum = 0.0
    lh_factor = [0.0]*k_br.value
    rh_factor = [0.0]*k_br.value
    
    for f in range(0, k_br.value):
        
        lh_factor[f] = np.dot(V_br.value[feats.indices][:,f], feats.values)  #KEY--this is used in v_grad matrix below
        rh_factor[f] = np.dot(V_br.value[feats.indices][:,f]**2, feats.values**2)
        
        #for i in feats.indices:
        #    i = int(i)
        #    lh_factor[f] += V_br.value[i][f]*feats[i]  #KEY--this is used in v_grad matrix below
        #    rh_factor += (V_br.value[i][f]**2) * (feats[i]**2)
        
        factor_sum += (lh_factor[f]**2 - rh_factor[f])
    factor_sum = 0.5 * factor_sum
    
    preProb = b_br.value + linear_sum + factor_sum
    
    prob = 1.0 / (1 + np.exp(-preProb))  #logit transformation
    
    
    #compute Gradients
    b_grad = label - prob
    w_grad = csr_matrix((b_grad*feats.values, feats.indices, np.array([0, feats.values.size])), 
                        (1, feats.size), dtype=np.float16)
    
    v_grad = csr_matrix((k_br.value, feats.size), dtype=np.float16)
    #for f in range(0, k_br.value):  # WORKING
    #    v_grad[feats.indices][f] = b_grad * (np.multiply(feats.values, lh_factor[f]) - np.multiply(V_br.value[feats.indices][f], feats[i]**2))
    
    
    
    return ([label, prob], [b_grad, w_grad, v_grad])

In [17]:
predRDD = vectorizedRDD.map(predict_grad).cache()
predRDD.take(1)

[([0, 0.9999999994788713],
  [-0.9999999994788713,
   <1x30946 sparse matrix of type '<class 'numpy.float16'>'
   	with 34 stored elements in Compressed Sparse Row format>,
   <2x30946 sparse matrix of type '<class 'numpy.float16'>'
   	with 0 stored elements in Compressed Sparse Row format>])]

In [23]:
lineT = predRDD.take(1)

In [26]:
lineT

[([0, 0.9999999994788713],
  [-0.9999999994788713,
   <1x30946 sparse matrix of type '<class 'numpy.float16'>'
   	with 34 stored elements in Compressed Sparse Row format>,
   <2x30946 sparse matrix of type '<class 'numpy.float16'>'
   	with 0 stored elements in Compressed Sparse Row format>])]

In [27]:
labelT, probT = lineT[0][0]
bGradT, wGradT, vGradT = lineT[0][1]

In [32]:
type(wGradT)

scipy.sparse.csr.csr_matrix

In [33]:
wGradT.indptr

array([ 0, 34], dtype=int32)

In [34]:
def get_item(row_index, column_index, matrix):
    # Get row values
    row_start = matrix.indptr[row_index]
    row_end = matrix.indptr[row_index + 1]
    row_values = matrix.data[row_start:row_end]

    # Get column indices of occupied values
    index_start = matrix.indptr[row_index]
    index_end = matrix.indptr[row_index + 1]

    # contains indices of occupied cells at a specific row
    row_indices = list(matrix.indices[index_start:index_end])

    # Find a positional index for a specific column index
    value_index = row_indices.index(column_index)

    if value_index >= 0:
        return row_values[value_index]
    else:
        # non-zero value is not found
        return 0

In [64]:
get_item(0, 4, wGradT)

-1.0

In [ ]:
nFailing = sc.accumulator(0)

In [134]:
from pyspark.accumulators import AccumulatorParam

class FloatAccumulatorParam(AccumulatorParam):
    """
    Custom accumulator for use in page rank to keep track of various masses.
    
    IMPORTANT: accumulators should only be called inside actions to avoid duplication.
    We stringly recommend you use the 'foreach' action in your implementation below.
    """
    def zero(self, value):
        return value
    def addInPlace(self, val1, val2):
        if type(val2) == list:
            val1[val2] += 1
        else:
            val1 += val2
        return val1

In [127]:
wGradAccumulators = np.array([sc.accumulator(0)]*num_feats)

In [130]:
wGradAccumulators = [sc.accumulator(0)]*num_feats

In [91]:
numNonZero_wGrad = sc.accumulator(np.zeros(num_feats))# i think this only needs to be calculated in the first training iteration, since the missing values won't change

TypeError: No default accumulator param for type <class 'numpy.ndarray'>

In [119]:
def getNumNonZero(line):
    wGradOnly = line[1][1]
    wGradNonZeros = wGradOnly.nonzero()[1].tolist()
    #print(len(wGradNonZeros))
    #numNonZero_wGrad[wGradNonZeros] += 1
    wGradAccumulators[wGradNonZeros[1]] += 1
    return wGradNonZeros

In [123]:
blah = predRDD.map(getNumNonZero)
blah.take(1)[0]

[0,
 1,
 2,
 4,
 5,
 7,
 10,
 20,
 32,
 122,
 154,
 175,
 214,
 364,
 369,
 495,
 504,
 626,
 634,
 855,
 1882,
 2080,
 2237,
 2281,
 2733,
 6170,
 10077,
 11284,
 14033,
 17114,
 19322,
 20016,
 25376,
 25620]

In [129]:
wGradAccumulators[blah.take(1)[0]].add(1)

AttributeError: 'numpy.ndarray' object has no attribute 'add'

In [ ]:
[wGradAccumulators[i] for i in blah.take(1)[0]]

In [131]:
for i in blah.take(1)[0]:
    wGradAccumulators[i].add(1)

In [132]:
wGradAccumulators

[Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulator<id=6, value=35>,
 Accumulat

In [78]:
numNonZero_wGrad = predRDD.map(lambda line: tuple(line[1][1].nonzero()[1].tolist()))# \
numNonZero_wGrad.take(1)[0]

(0,
 1,
 2,
 4,
 5,
 7,
 10,
 20,
 32,
 122,
 154,
 175,
 214,
 364,
 369,
 495,
 504,
 626,
 634,
 855,
 1882,
 2080,
 2237,
 2281,
 2733,
 6170,
 10077,
 11284,
 14033,
 17114,
 19322,
 20016,
 25376,
 25620)

In [73]:
wGradOnlyRDD = predRDD.map(lambda line: line[1][1])#.cache()# \
                    
                      #.reduce(lambda x,y: x+y)
                      #.mean()
wGradOnlyRDD.take(1)[0].nonzero()[1]

array([    0,     1,     2,     4,     5,     7,    10,    20,    32,
         122,   154,   175,   214,   364,   369,   495,   504,   626,
         634,   855,  1882,  2080,  2237,  2281,  2733,  6170, 10077,
       11284, 14033, 17114, 19322, 20016, 25376, 25620], dtype=int32)

In [ ]:
get_item(0, 3, wGradOnlyRDD.take(1)[0])

## Reduce sparse vectors, get mean of gradients, and update parameter vectors

In [ ]:
from scipy.sparse import vstack

In [ ]:
def _sgd(self, X, y, max_iter, batch_size, alpha=0.0005):
    """
    Runs Stochastic GD and logs error, weigths, gradient and 
    numerical gradient at every step

    Args:
        X(ndarray):      train objects
        y(ndarray):      answers for train objects
        max_iter(int):   number of EPOCHS, i.e., full passes over data
        batch_size(int): number of samples in one batch
        alpha(floar):    step size in direction of gradient
    Return:
        None
    """

    for epoch in range(max_iter):
        idxs = np.random.permutation(X.shape[0])
        X = X[idxs]
        y = y[idxs]
        for i in range(0, len(X), batch_size):

            self.history["coef"].append(self._theta[1:].copy())
            self.history["intercept"].append(self._theta[0].copy())

            rmse = self.score(X, y)
            self.history["cost"].append(rmse)

            # calculate gradient
            grad = self._grad(X[i:i + batch_size], y[i:i + batch_size])
            self.history["grad"].append(grad)

            # numerical gradient
            grad_num = self._grad_num(X[i:i + batch_size], y[i:i + batch_size])
            self.history["grad_num"].append(grad_num)

            # do gradient step
            self._theta -= alpha * grad